In [2]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

### 3.2.1 生成数据集

### 3.2.2 读取数据集

In [3]:
def data_iter(batch_size, features, labels):
    '''
    生成器函数，每次调用该函数时实现返回一个生成器对象，执行到yield返回该一个批次的数据，然后暂停执行、等待下一次迭代
    # yield 和 return 的区别
    1) yield 实现数据分批加载，每次返回一个批次的特征和标签，适合大规模数据集，按需加载、节省内存；
    2) return 一次执行并返回batch_size个数据
    '''
    num_examples = len(features)  # features.shape[0]
    indices = list(range(num_examples))
    random.shuffle(indices)  # 打乱样本顺序
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


### 3.2.3 初始化模型参数

In [4]:
w = torch.normal(0, 0.01, size = (2, 1), requires_grad = True)
b = torch.zeros(1, requires_grad=True)

### 3.2.4 定义模型

In [5]:
def linreg(X, w, b):
    '''线性回归模型'''
    return torch.matmul(X, w) + b  # 矩阵乘法、向量与标量加法

### 3.2.5 定义损失函数

In [6]:
def squared_loss(y_hat, y):
    '''平方损失函数'''
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

### 3.2.6 定义优化算法

In [7]:
def sgd(params, lr, batch_size):
    '''小批量随机梯度下降'''
    with torch.no_grad():  # 禁用梯度自动计算，因为这里我自己定义了梯度计算和更新的优化算法
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

### 3.2.7 训练

In [8]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss


In [ ]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()  # 标量才可以调用backward()
        sgd([w, b], lr, batch_size)
    
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)  # 监控该epoch时训练集上的损失大小
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

In [9]:
# 与预设的w, b 进行比较
print(f'w的估计误差： {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差： {true_b - b}')

NameError: name 'true_w' is not defined